
# Projet 2

In [31]:
# import des modules usuels
import numpy as np
import pandas as pd 

# options d'affichage
pd.set_option("display.min_rows", 16)

In [32]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)


**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [33]:
# Ajout d'une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
geo["CP Ville"]= geo["Code Postal"] +" "+ geo["Commune"]

In [34]:
# fonction recherche de ville
def ville(lat, long):
    R = 6371 # Rayon de la terre
    angle1 = np.radians(geo["Latitude"]-lat)
    angle2 = np.radians(geo["Longitude"]-long)
    a = np.sin(angle1/2)
    b= np.cos(geo["Latitude"])
    c= np.cos(lat)
    d = np.sin(angle2/2)
    distances = 2*R*np.arcsin(np.sqrt(a*a + b*c*d*d))
    cordville = np.argmin(distances)
    return geo["Commune"][cordville]

In [35]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [36]:
# on applique la fonction à une coordonnée tirée au hasard
# init du random
np.random.seed(0)

a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

'BOSJEAN'

In [37]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'PALAISEAU'

**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".
- Donner le décompte des valeurs accédées avec la clé "type".
- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
- Quelle commune est la plus complexe géométriquement ?
- Quelle commune est la seconde la plus complexe géométriquement ?
- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [80]:
# Utiliser la librairie Python json pour parser la colonne "geo_shape".
import json
geo["type"] = geo["geo_shape"].apply(lambda x: x.split(', ')[0])              
geo["coordinates"] = geo["geo_shape"].apply(lambda x: x.split(', ')[1])
geo

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,type,coordinates
Code INSEE,,,,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114,01400 L'ABERGEMENT-CLEMENCIAT,"{""type"": ""Polygon""","""coordinates"": [[[4.92627342692396"
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017,01640 L'ABERGEMENT-DE-VAREY,"{""type"": ""Polygon""","""coordinates"": [[[5.430089291292173"
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926,01500 AMBERIEU-EN-BUGEY,"{""type"": ""Polygon""","""coordinates"": [[[5.386190824025674"
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273,01330 AMBERIEUX-EN-DOMBES,"{""type"": ""Polygon""","""coordinates"": [[[4.895580369408855"
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320,01300 AMBLEON,"{""type"": ""Polygon""","""coordinates"": [[[5.614854625053272"
01007,01500,AMBRONAY,AIN,RHONE-ALPES,Commune simple,309.0,3359.0,2.3,"46.0055913782, 5.35760660735","{""type"": ""Polygon"", ""coordinates"": [[[5.413533...",46.005591,5.357607,01500 AMBRONAY,"{""type"": ""Polygon""","""coordinates"": [[[5.413533405897647"
01008,01500,AMBUTRIX,AIN,RHONE-ALPES,Commune simple,274.0,518.0,0.7,"45.9367134524, 5.3328092349","{""type"": ""Polygon"", ""coordinates"": [[[5.321986...",45.936713,5.332809,01500 AMBUTRIX,"{""type"": ""Polygon""","""coordinates"": [[[5.321986043355919"
01009,01300,ANDERT-ET-CONDON,AIN,RHONE-ALPES,Commune simple,294.0,696.0,0.3,"45.7873565333, 5.65788307924","{""type"": ""Polygon"", ""coordinates"": [[[5.656393...",45.787357,5.657883,01300 ANDERT-ET-CONDON,"{""type"": ""Polygon""","""coordinates"": [[[5.65639346945457"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [81]:
# Décompte des valeurs accédées avec la clé "type"
geo["type"].value_counts()

{"type": "Polygon"         36670
{"type": "MultiPolygon"       72
Name: type, dtype: int64

In [82]:
# Décompte des longueurs des listes accédées avec la clé "coordinates".
geo['coordinates'].str.len().value_counts()

35    27926
36     5642
34     2829
33      266
37       43
32       34
31        2
Name: coordinates, dtype: int64

In [83]:
# Commune est la plus complexe géométriquement 



In [85]:
# Villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2